In [1]:
import pandas as pd
import numpy as np
import time, config, traceback, re, os
from urllib.parse import quote_plus as urlparse
from pprint import pprint
from sqlalchemy import create_engine
import json
from distutils.log import debug
from flask import Flask, request, jsonify, make_response, send_file
from flask_cors import CORS, cross_origin
from numpy import asanyarray
from utils.func import *
import traceback
import logging as systemlog

_USER_ = config._USER_
_PASS_ = urlparse(config._PASS_)
_IP_ = config._IP_
_DB_NAME_ = config._DB_NAME_

con = f"mysql+mysqlconnector://{_USER_}:{_PASS_}@{_IP_}/{_DB_NAME_}"
engine = create_engine(con)

# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', None)

In [102]:
# Buat DataFrame untuk topping dan filling
topping_data = get_all_topping()
filling_data = get_all_fillings()

topping_data
# filling_data

,menu_id,menu_name,base_price,topping_id,topping_name,topping_price
0,1,Pizza,50000.0,10,Cheese,12000.0
1,1,Pizza,50000.0,11,Chicken,18000.0
2,1,Pizza,50000.0,12,Pepper,8000.0
3,2,Doughnut,20000.0,13,Blueberry,12000.0
4,2,Doughnut,20000.0,14,Cheese,12000.0
5,2,Doughnut,20000.0,15,Sugar Glaze,10000.0
6,3,Pie,45000.0,16,Pepper,8000.0
7,3,Pie,45000.0,17,Blueberry,12000.0
8,3,Pie,45000.0,18,Apple Slices,14000.0


In [16]:
# Mengambil data unik dari kolom menu_id, menu_name, dan base_price
list_menu = topping_data[['menu_id', 'menu_name', 'base_price']].drop_duplicates()
list_menu

,menu_id,menu_name,base_price
0,1,Pizza,50000.0
3,2,Doughnut,20000.0
6,3,Pie,45000.0


In [2]:
payload =   {
            "user": 2,
            "date": "2024-01-16",
            "orders": [
                {
                "menu_id": 1,
                "quantity": 2,
                "menu_name": "Pizza",
                "base_price": 50000,
                "toppings": [
                    {"topping_id": 10, "topping_name": "Cheese", "topping_price": 12000}
                ],
                "fillings": [{"filling_id": 3, "filling_name": "Tuna", "filling_price": 20000}]
                },
                {
                "menu_id": 2,
                "quantity": 1,
                "menu_name": "Doughnut",
                "base_price": 20000,
                "toppings": [
                    {"topping_id": 13, "topping_name": "Blueberry", "topping_price": 12000}
                ],
                "fillings": []
                },
                {
                "menu_id": 3,
                "quantity": 1,
                "menu_name": "Pie",
                "base_price": 45000,
                "toppings": [{"topping_id": 16, "topping_name": "Pepper", "topping_price": 8000}],
                "fillings": [
                    {"filling_id": 7, "filling_name": "Tuna", "filling_price": 20000}
                ]
                },
                {
                "menu_id": 2,
                "quantity": 2,
                "menu_name": "Doughnut",
                "base_price": 20000,
                "toppings": [{"topping_id": 15, "topping_name": "Sugar Glaze", "topping_price": 10000}],
                "fillings": [
                    {"filling_id": 4, "filling_name": "Apple Slices", "filling_price": 14000}
                ]
                }
            ]
            }

# payload

In [3]:
count_price(payload)

[2024-01-15 16:22:50 (+0700)] - data payload : {'user': 2, 'date': '2024-01-16', 'orders': [{'menu_id': 1, 'quantity': 2, 'menu_name': 'Pizza', 'base_price': 50000, 'toppings': [{'topping_id': 10, 'topping_name': 'Cheese', 'topping_price': 12000}], 'fillings': [{'filling_id': 3, 'filling_name': 'Tuna', 'filling_price': 20000}]}, {'menu_id': 2, 'quantity': 1, 'menu_name': 'Doughnut', 'base_price': 20000, 'toppings': [{'topping_id': 13, 'topping_name': 'Blueberry', 'topping_price': 12000}], 'fillings': []}, {'menu_id': 3, 'quantity': 1, 'menu_name': 'Pie', 'base_price': 45000, 'toppings': [{'topping_id': 16, 'topping_name': 'Pepper', 'topping_price': 8000}], 'fillings': [{'filling_id': 7, 'filling_name': 'Tuna', 'filling_price': 20000}]}, {'menu_id': 2, 'quantity': 2, 'menu_name': 'Doughnut', 'base_price': 20000, 'toppings': [{'topping_id': 15, 'topping_name': 'Sugar Glaze', 'topping_price': 10000}], 'fillings': [{'filling_id': 4, 'filling_name': 'Apple Slices', 'filling_price': 14000}]}

{'Status': 'Success',
 'Message': '',
 'orders': [{'menu_id': 1,
   'quantity': 2,
   'menu_name': 'Pizza',
   'base_price': 50000,
   'toppings': [{'topping_id': 10,
     'topping_name': 'Cheese',
     'topping_price': 12000}],
   'fillings': [{'filling_id': 3,
     'filling_name': 'Tuna',
     'filling_price': 20000}],
   'user': 2,
   'date': '2024-01-16',
   'total_price': 164000},
  {'menu_id': 2,
   'quantity': 1,
   'menu_name': 'Doughnut',
   'base_price': 20000,
   'toppings': [{'topping_id': 13,
     'topping_name': 'Blueberry',
     'topping_price': 12000}],
   'fillings': [],
   'user': 2,
   'date': '2024-01-16',
   'total_price': 32000},
  {'menu_id': 3,
   'quantity': 1,
   'menu_name': 'Pie',
   'base_price': 45000,
   'toppings': [{'topping_id': 16,
     'topping_name': 'Pepper',
     'topping_price': 8000}],
   'fillings': [{'filling_id': 7,
     'filling_name': 'Tuna',
     'filling_price': 20000}],
   'user': 2,
   'date': '2024-01-16',
   'total_price': 73000},
  {

In [139]:
df = pd.json_normalize(payload, "orders", ["user", "date"])

# Menghitung total harga setiap menu
df["total_price"] = (
    df["quantity"] * df["base_price"] +
    df["toppings"].apply(lambda x: sum(t.get("topping_price", 0) for t in x)) * df["quantity"] +
    df["fillings"].apply(lambda x: sum(f.get("filling_price", 0) for f in x)) * df["quantity"]
)

# Menampilkan hasil
print(df[["menu_id", "menu_name", "quantity", "total_price"]])

   menu_id menu_name  quantity  total_price
0        1     Pizza         2       164000
1        2  Doughnut         1        32000
2        3       Pie         1        73000
3        2  Doughnut         2        88000


In [141]:
total_order_price = df["total_price"].sum()
total_order_price

357000

In [143]:
json_data = payload
json_data["orders"] = df.to_dict(orient="records")
json_data["total_order_price"] = total_order_price

# Menampilkan hasil
print(json_data)

{'user': 2, 'date': '2024-01-16', 'orders': [{'menu_id': 1, 'quantity': 2, 'menu_name': 'Pizza', 'base_price': 50000, 'toppings': [{'topping_id': 10, 'topping_name': 'Cheese', 'topping_price': 12000}], 'fillings': [{'filling_id': 3, 'filling_name': 'Tuna', 'filling_price': 20000}], 'user': 2, 'date': '2024-01-16', 'total_price': 164000}, {'menu_id': 2, 'quantity': 1, 'menu_name': 'Doughnut', 'base_price': 20000, 'toppings': [{'topping_id': 13, 'topping_name': 'Blueberry', 'topping_price': 12000}], 'fillings': [], 'user': 2, 'date': '2024-01-16', 'total_price': 32000}, {'menu_id': 3, 'quantity': 1, 'menu_name': 'Pie', 'base_price': 45000, 'toppings': [{'topping_id': 16, 'topping_name': 'Pepper', 'topping_price': 8000}], 'fillings': [{'filling_id': 7, 'filling_name': 'Tuna', 'filling_price': 20000}], 'user': 2, 'date': '2024-01-16', 'total_price': 73000}, {'menu_id': 2, 'quantity': 2, 'menu_name': 'Doughnut', 'base_price': 20000, 'toppings': [{'topping_id': 15, 'topping_name': 'Sugar Gla

In [110]:
data = pd.DataFrame(payload["orders"])

# Merge dengan menu_df untuk mendapatkan base_price
orders_df = pd.merge(data, list_menu[['menu_id', 'base_price']], on='menu_id', how='left')

# Menghitung total price
orders_df['total_price'] = orders_df['base_price'] * orders_df['quantity']

orders_df

,menu_id,quantity,menu_name,toppings,fillings,base_price,total_price
0,1,2,Pizza,"[{'topping_id': 10, 'topping_name': 'Cheese'}]","[{'filling_id': 3, 'filling_name': 'Tuna'}]",50000.0,100000.0
1,2,1,Doughnut,"[{'topping_id': 13, 'topping_name': 'Blueberry'}]",[],20000.0,20000.0
2,3,1,Pie,"[{'topping_id': 16, 'topping_name': 'Pepper'}]","[{'filling_id': 7, 'filling_name': 'Tuna'}]",45000.0,45000.0


In [112]:
# Inisialisasi DataFrame toppings dari data JSON
toppings_df = pd.json_normalize(payload["orders"], 'toppings', ['menu_id', 'quantity'])

# Merge dengan topping_data untuk mendapatkan topping_price
toppings_df = toppings_df.merge(topping_data[['topping_id', 'topping_price']], on='topping_id', how='left')

# Menghitung total harga toppings
toppings_df['total_topping_price'] = toppings_df['topping_price'] * toppings_df['quantity']

toppings_df

,topping_id,topping_name,menu_id,quantity,topping_price,total_topping_price
0,10,Cheese,1,2,12000.0,24000.0
1,13,Blueberry,2,1,12000.0,12000.0
2,16,Pepper,3,1,8000.0,8000.0


In [113]:
# Inisialisasi DataFrame toppings dari data JSON
fillings_df = pd.json_normalize(payload["orders"], 'fillings', ['menu_id'])

# Merge dengan topping_data untuk mendapatkan topping_price
fillings_df = fillings_df.merge(filling_data[['filling_id', 'filling_price']], on='filling_id', how='left')

# Menghitung total harga toppings
fillings_df['total_filling_price'] = fillings_df['filling_price']

fillings_df

,filling_id,filling_name,menu_id,filling_price,total_filling_price
0,3,Tuna,1,20000.0,20000.0
1,7,Tuna,3,20000.0,20000.0


In [41]:
total_base_menu = orders_df['total_price'].sum()
total_base_menu

170000.0

In [105]:
total_topping_menu = toppings_df['total_topping_price'].sum()
total_topping_menu

36000.0

In [106]:
total_filling_menu = fillings_df['total_filling_price'].sum()
total_filling_menu

29000.0

In [44]:
total_price = total_base_menu + total_topping_menu + total_filling_menu
total_price

285000.0

In [87]:
def count(payload, type):
    ret = {'message' : 'failed'}
    count_list = []

    for order in payload["orders"]:
        menu_id = order["menu_id"]
        quantity = order["quantity"]

        for item in order[type]:
            count_list.append({
                'menu_id': menu_id,
                'quantity': quantity,
                'item_id': item[type[:-1] + '_id'],  # Jika 'type' adalah 'topping', ini menjadi 'topping_id'
                'item_name': item[type[:-1] + '_name'],  # Jika 'type' adalah 'topping', ini menjadi 'topping_name'
            })

    # Membuat DataFrame dari list
    count_df = pd.DataFrame(count_list)

    # Menambahkan pemeriksaan bahwa setiap order untuk suatu menu hanya memiliki satu jenis topping
    menu_counts = count_df.groupby(['menu_id', 'quantity']).size().reset_index(name='total_count')
    
    # Mengurutkan DataFrame berdasarkan menu_id dan quantity
    menu_counts = menu_counts.sort_values(by=['menu_id', 'quantity'])

    print(menu_counts)
    if (menu_counts['total_count'] > 1).any():
        ret['message'] = 'failed'
        return ret
    
    ret['message'] = 'success'
    return ret

In [96]:
cek_topping = count(payload, 'fillings')
cek_topping

   menu_id  quantity  total_count
0        1         1            1
1        1         2            1


{'message': 'success'}